In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pyCloudy as pc

In [ ]:
pc.config.cloudy_exe = 'cloudy.exe'

In [ ]:
# Create the directory 
models_dir = "../../SIGNALS_Models"
try: 
    os.makedirs(models_dir)
except OSError:
    if not os.path.isdir(models_dir):
        raise

In [ ]:
pc.print_make_file(models_dir)

In [ ]:
name = 'M1'
c_input = pc.CloudyInput('{0}/{1}'.format(models_dir, name))
c_input.set_BB(Teff = 50000., lumi_unit = 'q(H)', lumi_value = 48)
c_input.set_cste_density(2., ff = 1.)
c_input.set_radius(r_in = 15.)
c_input.set_abund(predef = 'ism')
c_input.set_distance(dist=1., unit='kpc', linear=True)
options = ('no molecules',
           'no level2 lines',
           'no fine opacities',
           'atom h-like levels small',
           'atom he-like levels small',
           'element limit off -7',
            )
c_input.set_other(options)
c_input.print_input()

In [ ]:
!cat ../../SIGNALS_Models/M1.in

In [ ]:
c_input.run_cloudy()

In [ ]:
pc.log_.level=3
M1 = pc.CloudyModel('{0}/{1}'.format(models_dir, name))
# A warning is issued because no emission line file is found. 
# This is a normal behaviour, we didn't set it.

In [ ]:
M1.print_stats()

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(12,4))
x = M1.depth/np.max(M1.depth)
axes[0].plot(x, M1.te)
axes[0].set_ylabel('Te [K]')
axes[1].plot(x, M1.get_ionic('O', 1))
axes[1].set_ylabel('O+/O')
axes[1].set_ylim(0,1)
axes[2].plot(x, M1.get_ionic('O', 2))
axes[2].set_ylabel('O++/O')
axes[2].set_ylim(0,1)
f.tight_layout()

In [ ]:
options = ('no molecules',
           'no level2 lines',
           'no fine opacities',
           'atom h-like levels small',
           'atom he-like levels small',
           'element limit off -7',
            )
from pyCloudy.utils.physics import abund_Asplund_2009

def get_R_in_Q0(logU, fr, NH=100, ff=1.0):
    c = pc.CST.CLIGHT
    U_mean = 10**logU
    w = (1 + fr**3.)**(1./3) - fr
    alpha_B = 2.6e-13
    Q0 = 4. * np.pi * c**3 * U_mean**3 / (3. * NH * ff**2 * alpha_B**2 * w**3)
    R_str = (3. * Q0 / (4 * np.pi * NH**2 * alpha_B * ff))**(1./3)
    R_in = fr * R_str
    return R_in, Q0

def make_mod(name, logU, ab_O):
    NH = 100
    ff = 0.1
    fr = 3.00
    R_in, Q0 = get_R_in_Q0(logU, fr=fr, NH=NH, ff=ff)
    c_input = pc.CloudyInput('{0}/{1}'.format(models_dir, name))
    c_input.set_BB(Teff = 50000., lumi_unit = 'q(H)', lumi_value = np.log10(Q0))
    c_input.set_cste_density(np.log10(NH), ff = ff)
    c_input.set_radius(r_in = np.log10(R_in))
    abund = abund_Asplund_2009.copy()
    delta_O = ab_O - 12 - abund['O']
    for elem in abund:
        if elem != 'He':
            abund[elem] += delta_O
    c_input.set_abund(ab_dict = abund)
    #c_input.set_grains('Orion {}'.format(10**delta_O))
    c_input.set_distance(dist=1., unit='kpc', linear=True)
    c_input.set_other(options)
    c_input.set_stop(('temperature off', 'pfrac 0.02'))
    c_input.set_emis_tab(['H  1  4861.33A', 'H  1  6562.81A', 
                          'N  2  6583.45A', 'O  3  5006.84A'])
    c_input.print_input()

In [ ]:
generic_name = 'M2_'

In [ ]:
for logU in np.arange(-3.5, -1.49, 0.5):
    for ab_O in np.arange(7, 9.1, 0.5):
        name = '{}{:.1f}_{:.1f}'.format(generic_name, logU, ab_O)
        make_mod(name, logU, ab_O)

In [ ]:
pc.run_cloudy(dir_=models_dir, n_proc=8, model_name=generic_name)

In [ ]:
pc.log_.level=2
Ms = pc.load_models('{}/{}'.format(models_dir, generic_name))
M = Ms[0]

In [ ]:
x = np.array([M.log_U_mean for M in Ms])
logHb = np.log10(np.array([M.get_emis_vol('H__1_486133A') for M in Ms]))
logQ = np.log10(np.array([M.Q.sum() for M in Ms]))
y = logHb - logQ
c = 12+np.array([M.abund['O'] for M in Ms])

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
sc = ax.scatter(x, y, c=c, edgecolor='none', s=50)
ax.set_xlabel('<logU>')
ax.set_ylabel(r'log L(H$\beta$)/Q$_ H$')
cb = f.colorbar(sc)
cb.set_label(r'12+log(O/H)')

In [ ]:
from pyCloudy.utils.astro import conv_arc

In [ ]:
# Define some parameters of the model:
model_name = 'model_4'
full_model_name = '{0}/{1}'.format(models_dir, model_name)
dens = 4. #log cm-3
Teff = 45000. #K
qH = 47. #s-1
r_min = 5e16 #cm
dist = 1.26 #kpc
# these are the commands common to all the models (here only one ...)
options = ('no molecules',
            'COSMIC RAY BACKGROUND',
            )
emis_tab = ['H  1  4861.33A',
            'H  1  6562.81A',
            'Ca B  5875.64A',
            'N  2  6583.45A',
            'O  1  6300.30A',
            'O  2  3726.03A',
            'O  2  3728.81A',
            'O  3  5006.84A',
            'BLND  4363.00A']
abund = {'He' : -0.92, 'C' : 6.85 - 12, 'N' : -4.0, 'O' : -3.40, 'Ne' : -4.00, 
         'S' : -5.35, 'Ar' : -5.80, 'Fe' : -7.4, 'Cl' : -7.00}
c_input = pc.CloudyInput(full_model_name)
c_input.set_BB(Teff = Teff, lumi_unit = 'q(H)', lumi_value = qH)
c_input.set_cste_density(dens)
# Defining the inner radius. A second parameter would be the outer radius (matter-bounded nebula).
c_input.set_radius(r_in=np.log10(r_min))
c_input.set_abund(ab_dict = abund, nograins = True)
c_input.set_other(options)
c_input.set_iterate() # (0) for no iteration, () for one iteration, (N) for N iterations.
c_input.set_sphere() # () or (True) : closed geometry, or (False): open geometry.
c_input.set_emis_tab(emis_tab) # better use read_emis_file(file) for long list of lines, where file is an external file.
c_input.set_distance(dist=dist, unit='kpc', linear=True) # unit can be 'kpc', 'Mpc', 'parsecs', 'cm'. If linear=False, the distance is in log.
c_input.print_input(to_file = True, verbose = False)


In [ ]:
# Running Cloudy with a timer. Here we reset it to 0.
pc.log_.timer('Starting Cloudy', quiet = True, calling = 'test1')
c_input.run_cloudy()
pc.log_.timer('Cloudy ended after seconds:', calling = 'test1')

In [ ]:
c_output = pc.CloudyModel(full_model_name)
c_output.print_stats()

In [ ]:
# define the size of the 3D cube and instanciate the object that manage it.
cube_size = 201
M_sphere = pc.C3D(c_output, dims=cube_size, center=True, n_dim=1)

In [ ]:
# plot the image of the OIII emission
f, ax = plt.subplots(figsize=(6,6))
ax.imshow(M_sphere.get_emis('O__3_500684A').sum(0));

In [ ]:
arcsec = lambda cm: conv_arc(dist=dist, dist_proj=cm)

In [ ]:
def make_mask(ap_center=[0., 0.], ap_size=[1., 1.]):
    """
    This returns a mask (values between 0. and 1.) to be multiplied to the image to take the flux passing through an aperture.
    An pc.C3D object named M_sphere must exist outside theis function
    """
    x_arc = arcsec(M_sphere.cub_coord.x_vec)
    y_arc = arcsec(M_sphere.cub_coord.y_vec)
    z_arc = arcsec(M_sphere.cub_coord.z_vec)
    X, Y = np.meshgrid(y_arc, x_arc)
    bool_mask = ((X > ap_center[0] - ap_size[0]/2.) & 
            (X <= ap_center[0] + ap_size[0]/2.) & 
            (Y > ap_center[1] - ap_size[1]/2.) & 
            (Y <= ap_center[1] + ap_size[1]/2.))
    mask = np.zeros_like(X)
    mask[bool_mask] = 1.0
    return mask

In [ ]:
# we define the mask. Can be change to see the effect of the aperture on line intensities
mask = make_mask(ap_center=[1.5, 2.3], ap_size=[50, 1.5])

In [ ]:
# Check that the mask is not empty
print(mask.size)
print(mask.sum())

In [ ]:
# We plot the OIII image and overplot the mask.
f, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(M_sphere.get_emis('O__3_500684A').sum(0), interpolation='None')
f.colorbar(im, ax=ax)
ax.contour(mask);

In [ ]:
# Hbeta is computed for the whole object and throught the aperture
Hb_tot = (M_sphere.get_emis('H__1_486133A')*M_sphere.cub_coord.cell_size).sum()
Hb_slit = ((M_sphere.get_emis('H__1_486133A')*M_sphere.cub_coord.cell_size).sum(1) * mask).sum()
print('{:.3e} {:.3e}'.format(Hb_tot, Hb_slit))

In [ ]:
# For every line, we compute the intensity for the whole object and throught the aperture.
# We also print out the difference due to the slit.
for label in M_sphere.m[0].emis_labels:
    I_tot = (M_sphere.get_emis(label).sum()*M_sphere.cub_coord.cell_size) / Hb_tot
    I_slit = ((M_sphere.get_emis(label).sum(1) * mask).sum()*M_sphere.cub_coord.cell_size) / Hb_slit
    print('line: {0:12s} I/Ib Total: {1:6.4f} I/Ib Slit: {2:6.4f} Delta: {3:4.1f}%'.format(label, I_tot, I_slit, 
                                                                                           (I_slit-I_tot)/I_tot*100))